## CSCI 470 Activities and Case Studies

1. For all activities, you are allowed to collaborate with a partner. 
1. For case studies, you should work individually and are **not** allowed to collaborate.

By filling out this notebook and submitting it, you acknowledge that you are aware of the above policies and are agreeing to comply with them.

# Supervised Learning

Supervised learning is broken up into two main types, _classification_ and _regression_. We'll go over a couple examples using $k$NN using scikit-learn (sklearn).

## scikit-learn API

We'll be using scikit-learn throughout the introductory portion of the class and so it's important to understand the basic API that models in that library implement. Models in scikit-learn are called _Estimators_ and the steps to use some of them are:

1. Initialize the model by setting its parameters `model = Model(param_1=val, param_2=val)`
1. Fit the model using `model.fit(features, labels)`
1. Predict using the model `model.predict(features)`


In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn as sk

%matplotlib inline
plt.style.use("ggplot")

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

We're going to start by making some synthetic data that we can use to train some models.

In [ ]:
X, y = make_blobs(n_samples=1000,random_state=0)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], marker='o', c=y, s=25, edgecolor='k')
plt.title("3 classes of data")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

Next we're going to split all our data into train and test sets then split the train set into model and validation.
We do so using sklearn's [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)
X_model, X_valid, y_model, y_valid = train_test_split(X_train, y_train, random_state=0, test_size=0.2)

print(f"All Data:        {len(X)} points")
print(f"Training data:   {len(X_train)} points")
print(f"Testing data:    {len(X_test)} points")
print(f"Modeling data:   {len(X_model)} points")
print(f"Validation data: {len(X_valid)} points")

Now we will load the models that we're going to use to fit this data. We will use sklearn's [$k$NN classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
k = 5

In [ ]:
# Step 1 - Initialize model with parameters
knn = KNeighborsClassifier(n_neighbors=k)
# Step 2 - Fit the model data
knn.fit(X_model, y_model)
# Step 3 - Predict the validation data
validationPredictions = knn.predict(X_valid)

Now that we have a model trained, we'll need to evaluate it. We'll load some classification metrics:

1. [Classification Report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)
1. [Confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)
1. [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score

In [ ]:
print(confusion_matrix(y_valid, validationPredictions))

In [ ]:
print(classification_report(y_valid,validationPredictions))

In [ ]:
f1_score(y_valid, validationPredictions, average="weighted")

Let's use the weighted F-1 score as the metric to measure how good a model is and use the validation set to determine the best value for `k` neighbors.

In [ ]:
def get_knn_training_scores(ks, model_features, model_labels):
    """Determine the f1-score of k values for kNN on a given data set
    Args:
        ks (int iterable): iterable of all the k values to apply
        model_features (iterable): the features from the model set to train on
        model_labels (iterable): the labels from the model set to train on
        
    Returns:
        dictionary: key is the k value and value is the weighted f1_score on the training set
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
ks = [1,2,3]
test_knn_train = get_knn_training_scores(ks, X_model, y_model)
assert list(test_knn_train.keys()) == ks
assert len(test_knn_train.keys()) == 3
assert np.array(list(test_knn_train.values())).all() <= 1
assert np.array(list(test_knn_train.values())).all() > 0

In [ ]:
def get_knn_validation_scores(ks, model_features, model_labels, validation_features, validation_labels):
    """Train a model on a dataset then return the F-1 score on another set
    Args:
        ks (int iterable): iterable of all the k values to apply
        model_features (iterable): the features from the model set to train on
        model_labels (iterable): the labels from the model set to train on
        validation_features (iterable): the features from the validation set to test on
        validation_labels (iterable): the labels from the validation set to test on
        
    Returns:
        dictionary: key is the k value and value is the weighted f1_score on the validation set
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
ks = [1,2,3]
test_knn_valid = get_knn_validation_scores(ks, X_model, y_model, X_valid, y_valid)
assert list(test_knn_train.keys()) == ks
assert len(test_knn_train.keys()) == 3
assert np.array(list(test_knn_train.values())).all() <= 1
assert np.array(list(test_knn_train.values())).all() > 0
assert test_knn_valid != test_knn_train

Now let's use these functions to determine the best value for $k$.

In [ ]:
ksToTest = [1,3,5,7,10,20,50,100]
training_scores = get_knn_training_scores(ksToTest, X_model, y_model)
validation_scores = get_knn_validation_scores(ksToTest, X_model, y_model, X_valid, y_valid)

Here, we'll plot the training and validation scores that were returned from the functions. 

In [ ]:
pd.Series(training_scores, name="Training").plot(kind="line")
pd.Series(validation_scores, name="Validation").plot(kind="line", label="Validation")
plt.legend()
plt.xlabel("k")
plt.ylabel("F1-score")
plt.show()

In [ ]:
# Select the best value for k and set it to bestK
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
clf = KNeighborsClassifier(bestK)
clf.fit(X_train, y_train)
testPredictions = clf.predict(X_test)

After selecting the best hyperparameter, we see how well it does on the test dataset as a sanity check. This is our final check after training the models to confirm that our validation was sufficient. 

In [ ]:
print("Confusion Matrix: \n")
print(confusion_matrix(y_test, testPredictions))
print("\n\nClassification Report:\n")
print(classification_report(y_test, testPredictions))

In [ ]:
assert f1_score(y_test, testPredictions, average="weighted") > 0.9

## Feedback

In [ ]:
def feedback():
    """Provide feedback on the contents of this exercise
    
    Returns:
        string
    """
    # YOUR CODE HERE
    raise NotImplementedError()